In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
from environment import *

from setting import *

In [ ]:
path_dict = path(
    TITLE,
    ELEMENTS,
    NMF_K,
    WT_HCC_K,
    H_HCC_K,
    UPLOAD_TO_PLOTLY,
)

In [ ]:
gps_map = ccal.load_gps_map(path_dict['gps_map_file_path'])

In [ ]:
feature_dicts = make_feature_dicts()

In [ ]:
with open('../code/highlight.json') as json_file:
    
    highlight = json.load(json_file)
    
highlight

In [ ]:
for wt_or_h, element_feature_dicts, element_highlight, element_marker_size in (
    (
        'h',
        feature_dicts['Sample'],
        highlight['Sample'],
        GPS_MAP_H_ELEMENT_MARKER_SIZE,
    ),
):
    
    for feature_group, features in element_highlight.items():
         
        if feature_group in element_feature_dicts:
            
            df = element_feature_dicts[feature_group]['df']
             
            annotation_type = element_feature_dicts[feature_group]['data_type']
            
            if features == 'All':
                
                features = df.index.tolist()
                
            for feature in features:
                
                if feature in df.index:
                    
                    if wt_or_h == 'w':
                        
                        elements = gps_map.w_elements
                        
                    elif wt_or_h == 'h':
                        
                        elements = gps_map.h_elements
                        
                    element_values = df.loc[feature, elements]
                    
                    if annotation_type == 'binary' and element_values.sum() < 3:
                        
                        continue
                        
                    title = '{}<br>{}'.format(
                        feature_group,
                        feature,
                    )
                    
                    maps_plotly_directory_path = path_dict['{}_map_plotly_directory_path'.format(wt_or_h)] 
                    
                    file_name = '{} {}.html'.format(
                        feature_group,
                        feature.replace(
                            '/',
                            '_',
                        ),
                    )
                    
                    if maps_plotly_directory_path is None:
                        
                        plotly_file_path = None
                        
                    else:
                        
                        plotly_file_path = '{}/{}'.format(
                            maps_plotly_directory_path,
                            file_name,
                        )
                        
                    gps_map.plot_gps_map(
                        wt_or_h,
                        annotation_x_element=element_values.to_frame().T,
                        annotation_std_maxs=None,
                        annotation_types=(annotation_type, ),
                        title=title,
                        element_marker_size=element_marker_size,
                        html_file_path='{}/{}'.format(
                            path_dict['{}_map_directory_path'.format(wt_or_h)],
                            file_name,
                        ),
                        plotly_file_path=plotly_file_path,
                    )
                    
                    if gps_map.h_element_states is not None:
                        
                        state_series = element_values.groupby(by=gps_map.h_element_states)
                        
                        names = tuple('State {}'.format(i) for i in state_series.groups.keys())
                        
                        if annotation_type == 'binary':
                            
                            ccal.plot_bar(
                                ((name, ) for name in names),
                                ((sum_, ) for sum_ in state_series.sum()),
                                names=names,
                                orientation='v',
                                title=title,
                                xaxis_title='State',
                                yaxis_title='N',
                            )
                            
                        else:
                            
                            ccal.plot_violin_or_box(
                                (series for group, series in state_series),
                                names=names,
                                violin_or_box='box',
                                title=title,
                            )